<a href="https://colab.research.google.com/github/Adithya-A-R/AI-Gym-Mate/blob/main/AI_GymMate_Pose_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Dataset & Loader
dataset = DummyPoseDataset()
loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PoseCNN().to(device)

# Loss & Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
epochs = 3

for epoch in range(epochs):
    total_loss = 0
    for images, keypoints in loader:
        images = images.to(device)
        keypoints = keypoints.to(device)

        preds = model(images)
        loss = criterion(preds, keypoints)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(loader):.4f}")


Epoch [1/3], Loss: 1.2037
Epoch [2/3], Loss: 0.0864
Epoch [3/3], Loss: 0.0892


In [5]:
class PoseCNN(nn.Module):
    def __init__(self):
        super(PoseCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 128x128

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64x64

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32
        )

        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 32 * 32, 512),
            nn.ReLU(),
            nn.Linear(512, 32)  # 16 keypoints × (x,y)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.regressor(x)
        return x


In [4]:
class DummyPoseDataset(Dataset):
    def __init__(self, num_samples=200):
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Fake image: 3 x 256 x 256
        image = torch.rand(3, 256, 256)

        # Fake keypoints: 16 keypoints → (x, y) → 32 values
        keypoints = torch.rand(32)  # normalized [0,1]

        return image, keypoints

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np

In [11]:
import cv2
import os

video_dir = "/content/AI_GymMate_Dataset/videos"
image_dir = "/content/AI_GymMate_Dataset/images"

os.makedirs(image_dir, exist_ok=True)

def extract_frames(video_name, label, frame_gap=5):
    video_path = os.path.join(video_dir, video_name)
    cap = cv2.VideoCapture(video_path)

    count = 0
    saved = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if count % frame_gap == 0:
            img_name = f"{label}_{saved:04d}.jpg"
            img_path = os.path.join(image_dir, img_name)
            cv2.imwrite(img_path, frame)
            saved += 1

        count += 1

    cap.release()
    print(f"{saved} frames extracted from {video_name}")

# Extract frames
extract_frames("squat.mp4", "squat")
extract_frames("pushup.mp4", "pushup")


536 frames extracted from squat.mp4
149 frames extracted from pushup.mp4
